# Preprocessing images

L'objectif est d'effectuer un processing sur les images afin d'avoir des tailles uniformes des fichiers images et enlever les pixels noirs (cropping)

In [6]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from PIL import Image

## Constantes

In [1]:

PATH_SRC = '/data/workspace_files/Ocular_dataset/ODIR_TrainingDataSet'
PATH_DST = '/data/workspace_files/Ocular_dataset/ODIR_TrainingDataSet_Preprocess'
IMAGE_WIDTH = 224

## Fonctions

In [2]:
def remove_black_pixels(folder_src, folder_dst, file):
    """
    Enlever le maximum de picel noir
    
    :param folder_src: Dossier source des images à modifier
    :param folder_dst: Dossier de destination des images modifiés
    :param file: Nom du fichier
    :return:
    """
    image = cv2.imread(join(folder_src, file))
    # Creation d'un masque avec les pixels colorés
    mask = image > 0

    # Coordonnes des pixels avec une couleur
    coordinates = np.argwhere(mask)

    # Definition du rectangle contenant les pixels colorés
    x0, y0, s0 = coordinates.min(axis=0)
    x1, y1, s1 = coordinates.max(axis=0) + 1  # slices are exclusive at the top

    # Decoupage de l'image selon le rectangle
    cropped = image[x0:x1, y0:y1]

    # Ecriture du fichier image dans le repertoire de destination
    cv2.imwrite(join(folder_dst, file), cropped)


In [1]:
def resize_image(folder_src, folder_dst, file, keep_ratio = False):
    """
        Redimensionner l'image
    """
    img = Image.open(join(folder_src, file))

    # Possibilité de garder le ratio
    if keep_ratio:
        width_perc = IMAGE_WIDTH/float(img.size[0])
        height_size = int((float(img.size[1]) * float(width_perc)))
        img = img.resize((IMAGE_WIDTH, height_size), Image.LANCZOS)
    else:
        img = img.resize((IMAGE_WIDTH, IMAGE_WIDTH), Image.LANCZOS)

    # Permutation de l'image droit/gauche de l'oei droit
    if 'right' in file:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)

    # sauvegarde et fermeture du fichier
    img.save(join(folder_dst, file), optimize = True, quality = 100)
    img.close()

In [4]:
def equalize_image(folder_src, folder_dst, file):
    """
        Equaliser l'histogramme de l'image
    """
    rgb_img = cv2.imread(join(folder_src,file))

    # conversion de l'image du format RGB au format YcrCb
    ycrcb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2YCrCb)

    # equalization sur le canal Y
    ycrcb_img[:, :, 0] = cv2.equalizeHist(ycrcb_img[:, :, 0])

    # conversion de l'image du format YcrCb au format RBG
    equalized_img = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(join(folder_dst,file), equalized_img)

# Preprocess Images

In [9]:
from tqdm import tqdm
if  not os.path.exists(PATH_DST):
    os.makedirs(PATH_DST)

files = [f for f in listdir(PATH_SRC) if isfile(join(PATH_SRC, f))]
for file in tqdm(files):
    #print(file)
    # cropping
    remove_black_pixels(PATH_SRC, PATH_DST, file)
    # equalize
    equalize_image(PATH_DST,PATH_DST, file)
    # resize
    resize_image(PATH_DST,PATH_DST, file)


  0%|          | 0/7000 [00:00<?, ?it/s]<ipython-input-3-db7b18350ae6>:10: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((IMAGE_WIDTH, IMAGE_WIDTH), Image.LANCZOS)
<ipython-input-3-db7b18350ae6>:14: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  img = img.transpose(Image.FLIP_LEFT_RIGHT)
100%|██████████| 7000/7000 [3:36:01<00:00,  1.85s/it]
